In [ ]:
# site = 'https://www.lawrencekenshin.com/'
# output = analyze(site, analyze_headings=True, analyze_extra_tags=True, follow_links=False)

In [4]:
import pandas as pd
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import nltk
from nltk.stem.snowball import SnowballStemmer
snow_stemmer = SnowballStemmer(language='english')
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
import csv

In [28]:
df = pd.read_csv("Table_data.csv")
df = df.drop(0, axis = 0)
df = df.drop(['Traffic source', 'Source type'], axis = 1)

In [29]:
df

,Source title,Views,Watch time (hours),Average view duration,Impressions,Impressions click-through rate (%)
1,muay thai,300401.0,16303.3729,0:03:15,0.0,NaN
2,muay thai vs taekwondo,124155.0,5109.4492,0:02:28,0.0,NaN
3,muay thai vs kickboxing,121324.0,8620.6183,0:04:15,0.0,NaN
4,lerdsila,113552.0,6735.1751,0:03:33,0.0,NaN
5,buakaw,106442.0,6056.4526,0:03:24,0.0,NaN
...,...,...,...,...,...,...
497,samart muay thai,1212.0,81.8400,0:04:03,0.0,NaN
498,taekwondo street fight,1208.0,56.5917,0:02:48,0.0,NaN
499,buakaw vs karate,1206.0,45.2700,0:02:15,0.0,NaN
500,mma fighter street,1197.0,31.4404,0:01:34,0.0,NaN


In [ ]:
df1 = df['Source title']
df1.dropna(inplace = True)
df1

In [ ]:
def stemmList(list):
    stemmed_list = []
    for l in list:
        words = l.split(" ")
        stem_words = []
        print(l)
        for word in words:
            x = snow_stemmer.stem(word)
            #x = porter_stemmer.stem(word)
            stem_words.append(x)
        key = " ".join(stem_words)
        print(key)
        stemmed_list.append(key)
    return stemmed_list
textlist = []
#queries.csv: your queries that should be clustered
textlist = df1.iloc(axis=0)[:].to_list()
labellist = textlist
textlist = stemmList(textlist)


In [ ]:
#-------------------------------------
LANGUAGE = 'english' # used for snowball stemmer
SENSITIVITY = 0.2 # The Lower the more clusters
MIN_CLUSTERSIZE = 2
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=10000,min_df=0.01, stop_words=LANGUAGE,use_idf=True, ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(textlist)
ds = DBSCAN(eps=SENSITIVITY, min_samples=MIN_CLUSTERSIZE).fit(tfidf_matrix)
clusters = ds.labels_.tolist()

cluster_df = pd.DataFrame(clusters, columns=['Cluster'])
keywords_df =  pd.DataFrame(labellist, columns=['Keyword'])
result = pd.merge(cluster_df, keywords_df, left_index=True, right_index=True)
grouping = result.groupby(['Cluster'])['Keyword'].apply(' | '.join).reset_index()
grouping.to_excel("clustered_queries.xlsx",index=False)